<a href="https://colab.research.google.com/github/Bayzid03/LangGraph-Hub/blob/main/Customer%20Support%20Agent/Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Still working on this project

Smart Customer Support Agent

In [ ]:
import os
from typing import TypedDict, Annotated, List
from langgraph.graph import StateGraph, END
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.runnables.graph import MermaidDrawMethod
from IPython.display import display, Image
from dotenv import load_dotenv

In [ ]:
load_dotenv()
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0.2)

Defining Agent State and Prompts

In [ ]:
class SupportAgentState(TypedDict):
    messages: Annotated[List[HumanMessage | AIMessage], "The messages in the conversation"]
    user_query: str
    query_category: str
    solution: str
    statisfaction_rating: int
    follow_up_needed: bool
    support_history: List[str]

query_analysis_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a customer support query analyzer. Analyze the customer query and categorize it into ONE of these categories:

Categories:
- technical: Software bugs, login issues, app crashes, feature problems, connectivity issues
- billing: Payment problems, subscription issues, refunds, charges, account billing
- general: Basic questions, account information, how-to guides, product information
- escalate: Complex issues, angry customers, legal matters, or anything requiring human intervention

Return ONLY the category name (technical/billing/general/escalate) without explanation."""),
    ("human", "Customer Query: {query}"),
])

technical_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a technical support specialist. Provide clear, step-by-step solutions for technical issues.

Format your response as:
**Problem Diagnosed:** [Brief description]
**Solution Steps:**
1. [Step 1]
2. [Step 2]
3. [Step 3]
**Additional Help:** [Any extra tips or resources]

Keep solutions practical and easy to follow."""),
    ("human", "Technical Issue: {query}"),
])

billing_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a billing support specialist. Help customers with payment and subscription issues.

Format your response as:
**Issue Summary:** [Brief description]
**Resolution Steps:**
1. [Step 1]
2. [Step 2]
3. [Step 3]
**Next Steps:** [What happens next or follow-up needed]

Be empathetic and provide clear billing information."""),
    ("human", "Billing Issue: {query}"),
])

general_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a friendly customer service representative. Provide helpful information for general inquiries.

Format your response as:
**Answer:** [Direct answer to their question]
**Additional Information:** [Helpful related details]
**Resources:** [Where they can find more help]

Be friendly, helpful, and informative."""),
    ("human", "Customer Question: {query}"),
])

escalation_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a customer service manager handling escalated issues. Be professional, empathetic, and assuring.

Format your response as:
**Acknowledgment:** [Acknowledge their concern]
**Next Steps:** [What will happen next]
**Timeline:** [When they can expect follow-up]
**Contact Information:** [How they can reach human support]

Show that their issue is being taken seriously."""),
    ("human", "Escalated Issue: {query}"),
])

satisfaction_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are evaluating customer satisfaction. Based on the solution provided and the original query, rate the likely satisfaction level from 1-5:

1 = Very Unsatisfied (complex issue, solution unclear)
2 = Unsatisfied (solution partially addresses issue)
3 = Neutral (solution is okay but could be better)
4 = Satisfied (good solution, addresses main concerns)
5 = Very Satisfied (excellent solution, comprehensive help)

Return ONLY the number (1-5) without explanation."""),
    ("human", "Original Query: {query}\n\nSolution Provided: {solution}"),
])

Define Node **Functions**

In [ ]:
def input_query(state: SupportAgentState) -> SupportAgentState:
    """Collect customer's support query"""
    print("=== Welcome to Smart Customer Support ===")
    print("Please describe your issue or question:")
    user_message = input("Your query: ")

    return {
        **state,
        "user_query": user_message,
        "messages": state['messages'] + [HumanMessage(content=user_message)],
        "support_history": state.get('support_history', []) + [f"Query: {user_message}"]
    }

def analyze_query(state: SupportAgentState) -> SupportAgentState:
    """Analyze and categorize the customer query using LLM"""
    print("🔍 Analyzing your query...")

    # Use LLM to categorize the query
    response = llm.invoke(query_analysis_prompt.format_messages(query=state['user_query']))
    category = response.content.strip().lower()

    print(f"📋 Query categorized as: {category.upper()}")
    print("🔄 Routing to appropriate support specialist...\n")

    return {
        **state,
        "query_category": category,
        "messages": state['messages'] + [AIMessage(content=f"Category: {category}")],
        "support_history": state['support_history'] + [f"Categorized as: {category}"]
    }

def technical_support(state: SupportAgentState) -> SupportAgentState:
    """Handle technical support issues"""
    print("🔧 Technical Support Specialist")
    print("=====================================")

    response = llm.invoke(technical_prompt.format_messages(query=state['user_query']))
    solution = response.content

    print(solution)
    print("\n" + "="*50 + "\n")

    return {
        **state,
        "solution": solution,
        "messages": state['messages'] + [AIMessage(content=solution)],
        "support_history": state['support_history'] + ["Technical solution provided"]
    }